In [ ]:
import kfp
from kfp import dsl
from kfp.components import create_component_from_func
# import kfp.components as components
from typing import NamedTuple

@create_component_from_func
def data_preprocessing():
  import numpy as np
  import pandas as pd
  from minio import Minio
  import numpy as np
  
  print("getting data")

  minio_client = Minio(
      "100.65.11.110:9000",
      access_key="v55j711ZNMtkTxky93Zk",
      secret_key="LjUSzTpY8PQa5AdM9DSDx3nSqNXUrLoTHLnThkfq",
      secure=False
  )
  minio_bucket = "sample"
  
  minio_client.fget_object(minio_bucket,"uci-secom.csv","/tmp/uci-secom.csv")
  
  data = np.load("/tmp/mycsv.csv")
  nv = pd.concat([data.isnull().sum(), 100 * data.isnull().sum()/data.shape[0]],axis=1).rename(columns={0:'Missing_Records', 1:'Percentage (%)'})
  df_na = nv[nv.Missing_Records>0].sort_values('Missing_Records', ascending=False)
  df_na = df_na[df_na["Percentage (%)"] > 50]
  data = data.drop(axis=1, columns=df_na.index)

  data.fillna(method='ffill', inplace=True)
  data.fillna(method='bfill', inplace=True)

  uni_col_list = []
  for column in data.columns:
    if data[column].nunique() == 1:
      uni_col_list.append(column)

  data_pre = data.drop(axis=1, columns=uni_col_list)
  
  np.save("/tmp/data_pre.csv",data_pre)
  minio_client.fput_object(minio_bucket,"data_pre","/tmp/data_pre.csv")

@create_component_from_func
def data_scaling():
  from imblearn.over_sampling import SMOTE
  from sklearn.model_selection import train_test_split
  from sklearn.preprocessing import StandardScaler
  from sklearn.decomposition import PCA
  from minio import Minio
  import numpy as np

  minio_client = Minio(
      "100.65.11.110:9000",
      access_key="minio",
      secret_key="minio123",
      secure=False
  )
  minio_bucket = "mlpipeline"
  
  minio_client.fget_object(minio_bucket,"data_pre.csv","/tmp/data_pre.csv")
  
  data = np.load("/tmp/data_pre.csv")

  X = data.iloc[:, 1:-1]
  y = data.iloc[:, -1]
  X_resample, y_resample  = SMOTE(random_state=1).fit_sample(X, y.values.ravel())
  X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample, random_state=50)

  sc= StandardScaler()
  X_sc = sc.fit_transform(X)
  X_train_sc = sc.fit_transform(X_train)
  X_test_sc = sc.transform(X_test)

  pca = PCA(0.85)
  X_sc_pca = pca.fit_transform(X_sc)

  X_train_pca = pca.transform(X_train_sc)
  X_test_pca = pca.transform(X_test_sc)
  
  # save to numpy file, store in Minio
  np.save("/tmp/X_train_pca.npy",X_train_pca)
  minio_client.fput_object(minio_bucket,"X_train_pca","/tmp/X_train_pca.npy")

  np.save("/tmp/X_test_pca.npy",X_test_pca)
  minio_client.fput_object(minio_bucket,"X_test_pca","/tmp/X_test_pca.npy")

  np.save("/tmp/y_train.npy",y_train)
  minio_client.fput_object(minio_bucket,"y_train","/tmp/y_train.npy")

  np.save("/tmp/y_test.npy",y_test)
  minio_client.fput_object(minio_bucket,"y_test","/tmp/y_test.npy")

@create_component_from_func
def data_modeling_xgb():
  from sklearn.model_selection import GridSearchCV
  from sklearn.externals import joblib
  from xgboost import XGBClassifier
  from minio import Minio
  import numpy as np  
  
  minio_client = Minio(
      "100.65.11.110:9000",
      access_key="minio",
      secret_key="minio123",
      secure=False)
  
  minio_bucket = "mlpipeline"
  
  minio_client.fget_object(minio_bucket,"X_train_pca.npy","/tmp/X_train_pca.npy")
  minio_client.fget_object(minio_bucket,"y_train.npy","/tmp/y_train.npy")
  
  X_train_pca = np.load("/tmp/X_train_pca.npy")
  y_train = np.load("/tmp/y_train.npy")

  xgb= XGBClassifier(eval_metric='error')
  params = {'max_depth':[2,4,6,8,10,12], 'random_state':[1]}

  grid_xgb = GridSearchCV(xgb, param_grid=params, scoring='accuracy', cv=10)
  grid_xgb.fit(X_train_pca, y_train)
  
  joblib.dump(grid_xgb, '/tmp/xgb_model.pkl') 
  
  minio_client.fput_object(minio_bucket,"xgb_model","/tmp/xgb_model.pkl")
  return "xgb"


@create_component_from_func
def data_modeling_randomforest():
  from minio import Minio
  from sklearn.externals import joblib
  import numpy as np  
  
  minio_client = Minio(
      "100.65.11.110:9000",
      access_key="minio",
      secret_key="minio123",
      secure=False)
  
  minio_bucket = "mlpipeline"
  
  minio_client.fget_object(minio_bucket,"X_train_pca.npy","/tmp/X_train_pca.npy")
  minio_client.fget_object(minio_bucket,"y_train.npy","/tmp/y_train.npy")
  
  X_train_pca = np.load("/tmp/X_train_pca.npy")
  y_train = np.load("/tmp/y_train.npy")
  
  from sklearn.ensemble import RandomForestClassifier
  grid_rf = RandomForestClassifier(n_estimators=60)
  grid_rf.fit(X_train_pca, y_train)

  joblib.dump(grid_rf, '/tmp/rf_model.pkl') 
  
  minio_client.fput_object(minio_bucket,"rf_model","/tmp/rf_model.pkl")
  return "rf"

@create_component_from_func
def data_modeling_svm():
  from sklearn.model_selection import GridSearchCV
  from sklearn.svm import SVC
  from sklearn.externals import joblib
  from minio import Minio
  import numpy as np  
  
  minio_client = Minio(
      "100.65.11.110:9000",
      access_key="minio",
      secret_key="minio123",
      secure=False)
  
  minio_bucket = "mlpipeline"
  
  minio_client.fget_object(minio_bucket,"X_train_pca.npy","/tmp/X_train_pca.npy")
  minio_client.fget_object(minio_bucket,"y_train.npy","/tmp/y_train.npy")
  
  X_train_pca = np.load("/tmp/X_train_pca.npy")
  y_train = np.load("/tmp/y_train.npy")
  
  svc= SVC(kernel='rbf')
  params = {'C':[0.01,0.1,1,10,100,1000], 'gamma':[0.01,0.1,1,10,100,1000]}

  grid_svm = GridSearchCV(svc, param_grid=params, scoring='accuracy', cv=10)
  grid_svm.fit(X_train_pca, y_train)

  joblib.dump(grid_svm, '/tmp/svm_model.pkl') 
  
  minio_client.fput_object(minio_bucket,"svm_model","/tmp/svm_model.pkl")
  return "svm"

@create_component_from_func
def model_evaluation(model: str) -> NamedTuple('Output', [('mlpipeline_metadata', 'Metadata'),('mlpipeline_metrics', 'Metrics')]):
  
  from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
  from minio import Minio
  from sklearn.externals import joblib
  import numpy as np  
  import pandas as pd
  import json
  
  minio_client = Minio(
      "100.65.11.110:9000",
      access_key="minio",
      secret_key="minio123",
      secure=False)
  
  minio_bucket = "mlpipeline" 
  
  model_name = model + "_model"
  model_path = "/tmp/" + model_name + ".pkl"
  
  minio_client.fget_object(minio_bucket,"X_test_pca.npy","/tmp/X_test_pca.npy")
  minio_client.fget_object(minio_bucket,"y_test.npy","/tmp/y_test.npy")
  minio_client.fget_object(minio_bucket, model_name, model_path)

  y_test = np.load("/tmp/y_test.npy")
  X_test_pca = np.load("/tmp/X_test_pca.npy")
  
  clf = joblib.load(model_path) 
  em = clf.best_estimator_
  pred = em.predict(X_test_pca)

  
  confusionmatrix = confusion_matrix(pred, y_test)
  model_accuracy = accuracy_score(pred, y_test)
  f1score = f1_score(y_test, pred)
  recallscore = recall_score(pred, y_test)
  precisionscore = precision_score(y_test, pred)

  confusionmatrix = confusionmatrix.numpy()
  vocab = list(np.unique(y_test))
  data = []
  for target_index, target_row in enumerate(confusion_matrix):
      for predicted_index, count in enumerate(target_row):
          data.append((vocab[target_index], vocab[predicted_index], count))

  df_cm = pd.DataFrame(data, columns=['target', 'predicted', 'count'])
  cm_csv = df_cm.to_csv(header=False, index=False)
  
  metadata = {
      "outputs": [
          {
              "type": "confusion_matrix",
              "format": "csv",
              "schema": [
                  {'name': 'target', 'type': 'CATEGORY'},
                  {'name': 'predicted', 'type': 'CATEGORY'},
                  {'name': 'count', 'type': 'NUMBER'},
                ],
              "target_col" : "actual",
              "predicted_col" : "predicted",
              "source": cm_csv,
              "storage": "inline",
              "labels": [0,1,2,3,4,5,6,7,8,9]
          }]
  }
    
  metrics = {
    'metrics': [{
        'name': 'model_accuracy',
        'numberValue':  float(model_accuracy),
        'format' : "PERCENTAGE"
      },{
        'name': 'f1_score',
        'numberValue':  float(f1score),
        'format' : "PERCENTAGE"
      },{
        'name': 'recall_score',
        'numberValue':  float(recallscore),
        'format' : "PERCENTAGE"
      },{
        'name': 'precision_score',
        'numberValue':  float(precisionscore),
        'format' : "PERCENTAGE"
      }]}
  

  from collections import namedtuple
  output = namedtuple('output', ['mlpipeline_metadata', 'mlpipeline_metrics'])
  return output(json.dumps(metadata), json.dumps(metrics))

@create_component_from_func
def model_version_saved():
  "saved model"
  
@create_component_from_func
def model_serving():
  """
  Create kserve instance
  """
  from kubernetes import client 
  from kserve import KServeClient
  from kserve import constants
  from kserve import utils
  from kserve import V1beta1InferenceService
  from kserve import V1beta1InferenceServiceSpec
  from kserve import V1beta1PredictorSpec
  from kserve import V1beta1TFServingSpec
  from datetime import datetime

  namespace = utils.get_default_target_namespace()

  now = datetime.now()
  v = now.strftime("%Y-%m-%d--%H-%M-%S")

  name='ml-pipline-{}'.format(v)
  kserve_version='v1beta1'
  api_version = constants.KSERVE_GROUP + '/' + kserve_version

  isvc = V1beta1InferenceService(api_version=api_version,
                                  kind=constants.KSERVE_KIND,
                                  metadata=client.V1ObjectMeta(
                                      name=name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                                  spec=V1beta1InferenceServiceSpec(
                                  predictor=V1beta1PredictorSpec(
                                      service_account_name="jiye-minio-kserve",
                                      tensorflow=(V1beta1TFServingSpec(
                                          storage_uri="s3://mlpipeline/models/ml-pipeline/")))))

  KServe = KServeClient()
  KServe.create(isvc)
  
# comp_data_preprocessing = components.create_component_from_func(data_preprocessing,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0")
# comp_data_scaling = components.create_component_from_func(data_scaling,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0")
# comp_data_modeling_xgb = components.create_component_from_func(data_modeling_xgb,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0")
# comp_data_modeling_randomforest = components.create_component_from_func(data_modeling_randomforest,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0")
# comp_data_modeling_svm = components.create_component_from_func(data_modeling_svm,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0")
# comp_model_evaluation = components.create_component_from_func(model_evaluation,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0")
# comp_model_version_saved = components.create_component_from_func(model_version_saved,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0")
# comp_model_serving = components.create_component_from_func(model_serving,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0",
#                                                            packages_to_install=['kserve==0.8.0.1'])

@dsl.pipeline(
  name='ml-pipeline',
  description='Semiconductor defect detection'
)
def ml_pipeline():
  step1 = data_preprocessing()  
  step2_1 = data_modeling_xgb()
  step2_2 = step2_1.after(step1)
  step3_1 = data_modeling_randomforest()
  step3_2 = step3_1.after(step1)
  step4_1 = data_modeling_randomforest()
  step4_2 = step4_1.after(step1)
  
  step5 = model_evaluation
  step5.after(step2_2)  
  step5.after(step3_2)  
  step5.after(step4_2)
    
  step6 = model_version_saved()
  step6.after(step5)
  
  step7 = model_serving()
  step7.after(step6)
    

if __name__ == '__main__':
    client = kfp.Client()
    kfp.compiler.Compiler().compile(ml_pipeline, 'ml_pipeline.yaml')